# Writing Viterbi Algorithm for the Primer

### Part A: Model Definition and Probability Calculation

Model parameters according to the Nature Primer:

**States:**
- E: Exon
- 5: 5′ Splice Site
- I: Intron

**Parameters:**
- Starting probabilities: Initial state probabilities
- Transition probabilities: Probabilities of moving between states
- Emission probabilities: Probabilities of observing symbols in each state

Calculate the log probability of a given path.

In [1]:
import math
import numpy as np

In [2]:
def log(x):
    return -float('inf') if x == 0 else math.log(x)

In [3]:
def path_log_probability(path, sequence, start_probs, trans_probs, emit_probs):
    if len(path) != len(sequence):
        raise ValueError("Path and sequence must have the same length")

    log_prob = 0.0

    for i in range(len(sequence)):
        current_state = path[i]
        current_symbol = sequence[i]
        if i == 0:
            log_prob += log(start_probs[current_state])
        else:
            prev_state = path[i-1]
            log_prob += log(trans_probs[prev_state].get(current_state, 0))

        log_prob += log(emit_probs[current_state].get(current_symbol, 0))

    final_state = path[-1]
    if final_state == 'I' and 'end' in trans_probs['I']:
        log_prob += log(trans_probs['I']['end'])

    return log_prob

In [4]:
hmm_states = ['E', '5', 'I']

hmm_start_probs = {
    'E': 1.0,
    '5': 0.0,
    'I': 0.0
}

hmm_trans_probs = {
    'E': {'E': 0.9, '5': 0.1},
    '5': {'I': 1.0},
    'I': {'I': 0.9, 'end': 0.1}
}

hmm_emit_probs = {
    'E': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
    '5': {'A': 0.05, 'C': 0.0, 'G': 0.95, 'T': 0.0},
    'I': {'A': 0.4, 'C': 0.1, 'G': 0.1, 'T': 0.4}
}



In [5]:
example_path = "EEEEEEEEEEEEEEEEEE5IIIIIII"
example_sequence = "CTTCATGTGAAAGCAGACGTAAGTCA"

log_prob = path_log_probability(
    example_path,
    example_sequence,
    hmm_start_probs,
    hmm_trans_probs,
    hmm_emit_probs
)

print(f"Log probability of the given path: {log_prob:.2f}")

Log probability of the given path: -41.22


### Part B: Implementation

Implementing the full Viterbi algorithm to find the most likely path for a given sequence.

1. **Initialization:** Set up the Viterbi table for the first position
2. **Recursion:** Fill the table by finding the most likely previous state for each current state
3. **Termination:** Find the most likely final state and trace back to find the best path

The algorithm uses dynamic programming to efficiently find the optimal path.

### Viterbi Algorithm Analysis

**Computational Complexity:**
- **Time Complexity:** O(T×N²) where T is sequence length and N is number of states
  - For each position (T), we check all possible transitions (N²)
- **Space Complexity:** O(T×N) to store the Viterbi matrix and backpointers

**Numerical Considerations:**
- We use log probabilities to prevent underflow with very small probability values
- This implementation handles special cases like impossible transitions

In [7]:
def viterbi_algorithm(sequence, states, start_probs, trans_probs, emit_probs):
    sequence_length = len(sequence)

    viterbi_matrix = [{} for _ in range(sequence_length)]

    backpointers = [{} for _ in range(sequence_length)]

    observed_symbol = sequence[0]
    for state in states:
        emission_prob = emit_probs[state].get(observed_symbol, 0)
        viterbi_matrix[0][state] = log(start_probs[state]) + log(emission_prob)
        backpointers[0][state] = None

    for t in range(1, sequence_length):
        observed_symbol = sequence[t]

        for current_state in states:
            emission_prob = emit_probs[current_state].get(observed_symbol, 0)

            if emission_prob == 0:
                continue

            best_prob = -float('inf')
            best_prev_state = None

            for prev_state in states:
                trans_prob = trans_probs.get(prev_state, {}).get(current_state, 0)
                if trans_prob > 0:
                    prev_prob = viterbi_matrix[t-1].get(prev_state, -float('inf'))
                    path_prob = prev_prob + log(trans_prob) + log(emission_prob)

                    if path_prob > best_prob:
                        best_prob = path_prob
                        best_prev_state = prev_state

            if best_prev_state is not None:
                viterbi_matrix[t][current_state] = best_prob
                backpointers[t][current_state] = best_prev_state

    final_position = sequence_length - 1
    best_final_prob = -float('inf')
    best_final_state = None

    for state in states:
        state_prob = viterbi_matrix[final_position].get(state, -float('inf'))

        if state == 'I' and 'end' in trans_probs['I']:
            state_prob += log(trans_probs['I']['end'])

        if state_prob > best_final_prob:
            best_final_prob = state_prob
            best_final_state = state

    best_path = [best_final_state] if best_final_state else []
    current_state = best_final_state

    for t in range(final_position, 0, -1):
        prev_state = backpointers[t].get(current_state)
        if prev_state is None:
            break
        best_path.insert(0, prev_state)
        current_state = prev_state

    return ''.join(best_path), best_final_prob

most_likely_path, log_probability = viterbi_algorithm(
    example_sequence,
    hmm_states,
    hmm_start_probs,
    hmm_trans_probs,
    hmm_emit_probs
)

print(f"Most likely path: {most_likely_path}")
print(f"Log probability: {log_probability:.2f}")

Most likely path: EEEEEEEEEEEEEEEEEEEEEEEEEE
Log probability: -38.68
